In [3]:
#import scipy.io
import numpy as np
#mnist = scipy.io.loadmat("C:\\Users\\User\\Desktop\\Internship\\mnist-original.mat")
#print(mnist["data"].shape)
#print(mnist["target"].shape)

from sklearn.datasets import load_iris
iris = load_iris()
#iris["data"] = iris["data"].T
print(iris["data"].shape)
print(iris["target"].shape)

(150, 4)
(150,)


In [4]:
from sklearn.model_selection import train_test_split

In [6]:
# X_train + validation training set (60000, 784)
# X_test test set (10000, 784)
# y_train_val labels of the train set (60000,) (vector)
# y_test labels of the test set 
#First, divide the data set into (train+validation) and test. Then devide (train+val) into train and val 
#train 50000, test 100000, val 100000
X_train_val, X_test, y_train_val, y_test = train_test_split(
    iris["data"], iris["target"], test_size=20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=20, random_state=42)

#print(X_train_val)
#print(X_test)
#print("Labels")
#print(y_train_val)
#print(y_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [14]:
#apriori probabilities for each number 
apriori_array = np.zeros([4,1])
for i in range(0,4):
    apriori_array[i] = np.prod(iris["target"][iris["target"]==i].shape)
    
apriori_array = apriori_array / np.prod(iris["target"].shape)
print(apriori_array)

[[ 0.33333333]
 [ 0.33333333]
 [ 0.33333333]
 [ 0.        ]]


In [21]:
random_forest_clf = RandomForestClassifier(random_state=42)
extra_trees_clf = ExtraTreesClassifier(random_state=42)
#svm_clf = LinearSVC(random_state=42,)
mlp_clf = MLPClassifier(random_state=42, max_iter=800)
#gaussianNB_clf = GaussianNB(apriori_array)

In [22]:
print(X_train.shape)
print(y_train.shape)

(110, 4)
(110,)


In [24]:
estimators = [random_forest_clf, extra_trees_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
Training the MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_lay

In [25]:
##score() method measures the quality of the predictions given a test set
# где модели делают предикшены? когда вызывается .score()
value = []
print("Quality of the predictions:")
[estimator.score(X_val, y_val) for estimator in estimators]

for estimator in estimators:
    print(estimator.score(X_val, y_val))
    
print("Weights:")
for estimator in estimators:
    exp_weight=np.exp(10*estimator.score(X_val, y_val))
    value.append(np.exp(10*estimator.score(X_val, y_val)))
print(value)
sum_value=sum(value)
#print("Normalized weights:")
#for estimator in estimators:
    #exp_weight=np.exp(10*estimator.score(X_val, y_val))/sum_value
    #print(exp_weight)

Quality of the predictions:
0.85
0.9
0.95
Weights:
[4914.7688402991344, 8103.0839275753842, 13359.726829661873]


#print missclassified elements     

for estimator in estimators:
    print(y_val[estimator.predict(X_val) != y_val])
    print('____________________________')

# Stacking of first level learners

In [26]:
normalized_weights = (value/sum_value)
print(normalized_weights)

[ 0.18632372  0.30719589  0.50648039]


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
#predict_proba(X_val) returns probability estimates for the test vector X, e.g. predcit elements 0-9 and provide its probability
#predict computes argmax of  predic_prob
log_clf = LogisticRegression(random_state=42)
i = 0
for estimator in estimators:
    prob_for_est = estimator.predict_proba(X_val) 
    prob_for_est *= normalized_weights[i]
    print(prob_for_est.shape)
    if i == 0:
        X_train2 = prob_for_est
    else:
        X_train2 = np.hstack((X_train2, prob_for_est))
    i+=1
    
#print(X_train2.shape)
    
y_train2 = y_val
    
log_clf.fit(X_train2, y_train2)

score = cross_val_score(log_clf, X_train2, y_train2, cv=3, verbose=3)
mean_score = score.mean()
print("Mean score for training of the 2nd level learner = %f" % mean_score)


(20, 3)
(20, 3)
(20, 3)
[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.8333333333333334, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.8333333333333334, total=   0.0s
Mean score for training of the 2nd level learner = 0.888889


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


In [28]:
i=0
for estimator in estimators:
    prob_for_est = estimator.predict_proba(X_test) 
    prob_for_est *= normalized_weights[i]
    #print(prob_for_est.shape)
    if i == 0:
        X_test2 = prob_for_est
    else:
        X_test2 = np.hstack((X_test2, prob_for_est))
    i+=1
    
#print(X_train2.shape)
    
y_test2 = y_test
    
#log_clf.predict(X_test2, y_test2)

score = cross_val_score(log_clf, X_test2, y_test2, cv=3, verbose=3)
mean_score = score.mean()
print("Mean score for test set of the 2nd level learner = %f" % mean_score)


[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.8571428571428571, total=   0.0s
[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.0s
Mean score for test set of the 2nd level learner = 0.952381


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


# Bagging

In [42]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=4,
     bootstrap=True, n_jobs=-1, random_state=42) #max_samples=100,
bag_clf.fit(X_train, y_train)
#y_pred = bag_clf.predict(X_test)
#score = cross_val_score(bag_clf, X_train2, y_train2, cv=3, verbose=3)
#mean_score = score.mean()
#print("Mean score for training of the 2nd level learner = %f" % mean_score)
score = cross_val_score(bag_clf, X_test, y_test, cv=3, verbose=3)
mean_score = score.mean()
print("Mean score for test set of the 2nd level learner = %f" % mean_score)


[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   2.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


[CV] ...................................... , score=1.0, total=   2.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.6s remaining:    0.0s


[CV] ...................................... , score=1.0, total=   2.8s
Mean score for test set of the 2nd level learner = 1.000000


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.5s finished


# Boosting AdaBoost

In [33]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)
score = cross_val_score(ada_clf, X_test, y_test, cv=3, verbose=3)
mean_score = score.mean()
print("Mean score for test set of the 2nd level learner = %f" % mean_score)

[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ...................................... , score=1.0, total=   0.1s
Mean score for test set of the 2nd level learner = 1.000000


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s finished


# Voting Classifier

In [34]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    #("gaussianNB_clf", gaussianNB_clf),
    ("mlp_clf", mlp_clf),
]
voting_clf = VotingClassifier(named_estimators)

In [35]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
   ...       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [36]:
voting_clf.score(X_val, y_val)

0.90000000000000002

# Voting Classifier SOFT

In [37]:
voting_clf.voting = "soft"

In [38]:
voting_clf.score(X_val, y_val)

0.90000000000000002

In [39]:
voting_clf.score(X_test, y_test)

1.0